In [ ]:
# Librerias utiles
import pandas as pd 
import numpy as np 
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from mapsmx import MapsMX
import geopandas
import pyproj
import json
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [ ]:
app = dash.Dash()

In [ ]:
state = MapsMX().get_geo('state')
muns = MapsMX().get_geo('municipality')
yuc = muns[muns['cve_ent']=='31']

In [ ]:
conapo = pd.read_csv('conapo.csv')

In [ ]:
servicios = pd.read_csv('../BD-sucias/servicios_18_OCTUBRE_2021.csv', low_memory=False)
servicios.fecha_captura = pd.to_datetime(servicios.fecha_captura, format='%d/%m/%Y', errors = 'ignore')

In [ ]:
semujeres = servicios[servicios.dependenciaquebrindoservicio=='SECRETARÍA DE LAS MUJERES'].index
prodenay = servicios[servicios.dependenciaquebrindoservicio=='PROCURADURÍA DE LA DEFENSA DEL MENOR Y LA FAMILIA'].index
ssp = servicios[servicios.dependenciaquebrindoservicio=='SECRETARÍA DE SEGURIDAD PÚBLICA'].index
ceeav = servicios[servicios.dependenciaquebrindoservicio=='COMISIÓN EJECUTIVA ESTATAL DE ATENCIÓN A VÍCTIMAS'].index
segey = servicios[servicios.dependenciaquebrindoservicio== 'SECRETARÍA DE EDUCACIÓN DEL GOBIERNO DEL ESTADO DE YUCATÁN'].index
servicios.loc[semujeres, 'dependenciaquebrindoservicio'] = 'SEMUJERES'
servicios.loc[prodenay, 'dependenciaquebrindoservicio'] = 'PRODENNAY'
servicios.loc[ssp, 'dependenciaquebrindoservicio'] = 'SSP'
servicios.loc[ceeav, 'dependenciaquebrindoservicio'] = 'CEEAV'
servicios.loc[segey, 'dependenciaquebrindoservicio'] = 'SEGEY'

In [ ]:
selecciones = servicios[servicios.tiposervicio=='Seleccione'].index
servicios.loc[selecciones, 'tiposervicio'] = 'No identificado'

In [ ]:
victimas = pd.read_csv('reportes/reporte_semujeres_18_OCTUBRE_2021.csv',low_memory=False, dtype={'pk_perfil_agresor': 'object','num_hijos':'int32'}, parse_dates=['fecha_recepcion','fecha_hechos'])

In [ ]:
subtipo = pd.read_csv('../BD-sucias/subtipo_18_OCTUBRE_2021.csv', low_memory=False)
subtipo.fecha_recepcion = pd.to_datetime(subtipo.fecha_recepcion, format='%d/%m/%Y', errors='ignore')
subtipo.fecha_hechos = pd.to_datetime(subtipo.fecha_hechos, format='%d/%m/%Y', errors='ignore')
subtipo = subtipo.dropna()
selecciones = subtipo[subtipo.SubtipoOrd=='Seleccione'].index
subtipo.loc[selecciones, 'SubtipoOrd'] = 'No especificado'

In [ ]:
victimas = pd.concat([victimas, victimas.tipos.str.get_dummies(sep=',')], axis=1)

In [ ]:
victimas['Feminicida']=0
fems = victimas[victimas['descripcion_otro_tipos']=='FEMINICIDA'].index
victimas.loc[fems,'Feminicida'] = 1
victimas.loc[fems,'Otro'] = 0

In [ ]:
feminicidios = pd.read_csv('../BD-sucias/feminicidios-corregida.csv', low_memory=False)
feminicidios.fecha_recepcion = pd.to_datetime(feminicidios.fecha_recepcion, format='%d/%m/%Y', errors='ignore')
feminicidios['año_recepcion'] = feminicidios['fecha_recepcion'].dt.year
nas = feminicidios[(feminicidios.TipoRelacion.isna()==True)|(feminicidios.TipoRelacion=='Seleccione')].index
feminicidios.loc[nas,'TipoRelacion'] = 'Desconocido'
com = feminicidios[(feminicidios.TipoRelacion=='En la comunidad')].index
feminicidios.loc[com,'TipoRelacion'] = 'Comunidad'

In [ ]:
feminicidios = pd.concat([feminicidios, feminicidios.tipos.str.get_dummies(sep=',')], axis=1)

In [ ]:
feminicidios['Feminicida']=0
fems = feminicidios[feminicidios['descripcion_tipo_violencia'].str.contains('FEM')].index
feminicidios.loc[fems,'Feminicida'] = 1
feminicidios.loc[fems,'Otro'] = 0

In [ ]:
import re
def mayusculas(s):
    return ' '.join([i.strip() for i in re.findall('[A-Z][^A-Z]*', s)])
def numeros(n):
    return ', '.join(n.split('.0'))
def tokenize(s):
    temp = s.sub(',','')
    return ', '.join(temp)
def unir_unicos(s):
    return ', '.join(set(s))
def unir_resto(s):
    return ', '.join(set(s.split(' ')))

In [ ]:
fechas_groupby = pd.DataFrame(victimas.groupby('fecha_hechos')['pk_caso'].count())
fechas_groupby = fechas_groupby.rename(columns={'pk_caso':'Casos registrados'})
fechas_groupby['var'] = round(fechas_groupby['Casos registrados'].pct_change()*100,2)
fechas_groupby = pd.DataFrame(fechas_groupby[fechas_groupby.index >= '1990-01-01'])
semanas_groupby = pd.DataFrame(fechas_groupby.reset_index().groupby(pd.Grouper(freq='W', key='fecha_hechos'))['Casos registrados'].sum()).reset_index()
semanas_groupby['var'] = round(semanas_groupby['Casos registrados'].pct_change()*100,2)
inf = semanas_groupby[semanas_groupby['var'].isin([np.inf, -np.inf])].index
#semanas_groupby.loc[inf,'var']=semanas_groupby.loc[inf,'Casos registrados']*100
semanas_groupby.loc[inf,'var']=100
inf = semanas_groupby[semanas_groupby['var'].isin([np.nan])].index
#semanas_groupby.loc[inf,'var']=semanas_groupby.loc[inf,'Casos registrados']*100
semanas_groupby.loc[inf,'var']=0
fig_9 = px.line(semanas_groupby, x=semanas_groupby.fecha_hechos, y="Casos registrados", title='Grafica 9. Variación de casos de violencia hacia las mujeres',
                hover_data={"var": True}, labels={'fecha_hechos':'Fecha Hechos'},markers=True,
                range_x=['2015-01-01',max(semanas_groupby.fecha_hechos)],
                #range_y=[0,500]
                )
fig_9.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            #dict(step="all")
        ])
    )
)
fig_9.update_traces(hovertemplate='Fecha=%{x}<br>Casos registrados=%{y}<br>Variación=%{customdata[0]}%<extra></extra>')

In [ ]:
fechas_groupby = subtipo.groupby(['fecha_hechos','SubtipoOrd'])['fk_euv'].count()
fechas_groupby=pd.DataFrame(fechas_groupby.reset_index())
fechas_groupby_emer = fechas_groupby[fechas_groupby.SubtipoOrd=='Emergencia']
fechas_groupby_na = fechas_groupby[fechas_groupby.SubtipoOrd=='No especificado']
fechas_groupby_prev = fechas_groupby[fechas_groupby.SubtipoOrd=='Preventiva']
fechas_groupby_total = subtipo.groupby(['fecha_hechos'])['fk_euv'].count()
fechas_groupby_total=pd.DataFrame(fechas_groupby_total.reset_index())
total = pd.DataFrame(fechas_groupby_total.reset_index().groupby([pd.Grouper(freq='W', key='fecha_hechos')])['fk_euv'].sum()).reset_index()
total['var'] = round(total['fk_euv'].pct_change()*100,2)
inf = total[total['var'].isin([np.inf, -np.inf])].index
total.loc[inf,'var']=100
nan = total[total['var'].isin([np.nan])].index
total.loc[nan,'var']=0
emer = pd.DataFrame(fechas_groupby_emer.reset_index().groupby([pd.Grouper(freq='W', key='fecha_hechos')])['fk_euv'].sum()).reset_index()
emer['var'] = round(emer['fk_euv'].pct_change()*100,2)
inf = emer[emer['var'].isin([np.inf, -np.inf])].index
emer.loc[inf,'var']=100
nan = emer[emer['var'].isin([np.nan])].index
emer.loc[nan,'var']=0
prev = pd.DataFrame(fechas_groupby_prev.reset_index().groupby([pd.Grouper(freq='W', key='fecha_hechos')])['fk_euv'].sum()).reset_index()
prev['var'] = round(prev['fk_euv'].pct_change()*100,2)
inf = prev[prev['var'].isin([np.inf, -np.inf])].index
prev.loc[inf,'var']=100
nan = prev[prev['var'].isin([np.nan])].index
prev.loc[nan,'var']=0
na = pd.DataFrame(fechas_groupby_na.reset_index().groupby([pd.Grouper(freq='W', key='fecha_hechos')])['fk_euv'].sum()).reset_index()
na['var'] = round(na['fk_euv'].pct_change()*100,2)
inf = na[na['var'].isin([np.inf, -np.inf])].index
na.loc[inf,'var']=100
nan = na[na['var'].isin([np.nan])].index
na.loc[nan,'var']=0
fig_10 = go.Figure()
fig_10.add_trace(go.Scatter(
        x=na.fecha_hechos,
        y=na.fk_euv,
        name='No especificado',
        text=na['var'],
        mode='markers+lines'
        ))
fig_10.add_trace(go.Scatter(
        x=emer.fecha_hechos,
        y=emer.fk_euv,
        name='Emergencia',
        text=emer['var'],
        mode='markers+lines'
        ))
fig_10.add_trace(go.Scatter(
        x=prev.fecha_hechos,
        y=prev.fk_euv,
        name='Preventiva',
        text=prev['var'],
        mode='markers+lines'
        ))

fig_10.add_trace(go.Scatter(
        x=total.fecha_hechos,
        y=total.fk_euv,
        name='Total',
        text=total['var'],
        mode='markers+lines'
        ))

fig_10.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            #dict(step="all")
        ])
    )
)

#print("plotly express hovertemplate:", fig.data[0].hovertemplate)
fig_10.update_traces(hovertemplate='Fecha hecho=%{x}<br>Órdenes registradas=%{y}<br>Variación=%{text}%<extra></extra>',)
fig_10.update_xaxes(range=['2015-01-01',max(semanas_groupby.fecha_hechos)])
fig_10.update_layout(
    title="Grafica 10. Variación de órdenes de proteccion por tipo",xaxis_title="Fecha Hechos", yaxis_title="Órdenes de Protección")

In [ ]:
app.layout = html.Div(
    children=[
        html.H1(children="INDICADORES MUNICIPALES TABLERO",),
        html.H3(
            children="Casos totales: {}".format(len(victimas)),
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(
                            children="Date Range",
                            className="menu-title"
                            ),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed=victimas.fecha_recepcion.min(),
                            max_date_allowed=victimas.fecha_recepcion.max(),
                            start_date=victimas.fecha_recepcion.min(),
                            end_date=victimas.fecha_recepcion.max(),
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(
                            children="Tasa de violencia municipal (año)",
                            className="menu-title"
                            ),
                        dcc.Input(
                            id="tasa",
                            type='number',
                            placeholder="Año",
                            value=2021,
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Tipo de violencia",
                            className="menu-title"
                            ),
                        dcc.Dropdown(
                            id="violencia-tipo",
                            options=[
                                {"label": dep, "value": dep}
                                for dep in ['Tasa 1000 Total','Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro']
                            ],
                            value='Tasa 1000 Total',
                            multi=False,
                            clearable=False,
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        dcc.Graph(
            id = 'map_tasa',  
        ),
        dcc.Graph(
            id = 'fig_1',  
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(
                            children="Tipo de sentencia",
                            className="menu-title"
                            ),
                        dcc.Dropdown(
                            id="sentencia-tipo",
                            options=[
                                {"label": dep, "value": dep}
                                for dep in ['Feminicidios','Tentativas']
                            ],
                            value='Feminicidios',
                            multi=False,
                            clearable=False,
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        dcc.Graph(
            id = 'map_1',  
        ),
        dcc.Graph(
            id = 'fig_2',  
        ),
        dcc.Graph(
            id = 'fig_3',  
        ),
        dcc.Graph(
            id = 'fig_4',  
        ),
        dcc.Graph(
            id = 'fig_5',  
        ),
        dcc.Graph(
            id = 'map_2',  
        ),
        dcc.Graph(
            id = 'fig_6',  
        ),
        dcc.Graph(
            id = 'fig_7',  
        ),
        dcc.Graph(
            id = 'map_3',  
        ),
        dcc.Graph(
            id = 'fig_8',  
        ),
        dcc.Graph(
            id = 'fig_9', figure=fig_9 
        ),
        dcc.Graph(
            id = 'fig_10',figure=fig_10  
        ),
    ]
)

In [ ]:
feminicidios_copia = feminicidios

In [ ]:
conapo_copia=conapo

In [ ]:
@app.callback(
    [   
        Output('fig_1', "figure"),
        Output('fig_2', "figure"),
        Output('fig_3', "figure"),
        Output('fig_4', "figure"),
        Output('fig_5', "figure"),
        Output('fig_6', "figure"),
        Output('fig_7', "figure"),
    ],    
    [
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ]
)
def update_charts(start_date, end_date):
    mask = (
        (victimas.fecha_recepcion >= start_date)
        & (victimas.fecha_recepcion <= end_date)
    )
    feminicidios = feminicidios_copia
    fem_mask = (
        (feminicidios.fecha_recepcion >= start_date)
        & (feminicidios.fecha_recepcion <= end_date)
    )
    

    filtered_data = victimas.loc[mask, :]
    filtered_fems = feminicidios.loc[fem_mask, :]

    # Tipos de violencia
    tipos = filtered_data.iloc[:,[i for i in range(42,49)]]
    tipos = pd.DataFrame(tipos.sum())
    tipos = tipos.sort_values(0,ascending=False)
    

    # fig 1 - violencias tipos
    tipos = tipos.sort_values(0, ascending=False)
    y = tipos[0].values
    y_total = len(victimas)
    fig_1 = px.bar(x=tipos.index,
                y=tipos[0],
                text= np.round(y/y_total*100,2),
                labels = {'x': 'Tipo de violencia', "y":'Número de casos', 'text':'Porcentaje'},
                color=px.colors.qualitative.Prism[:7],
                color_discrete_map="identity",
                title='Gráfica 1. Casos de violencia hacia las mujeres por tipo de violencia'
                )
    fig_1.update_xaxes(type='category')
    fig_1.update_layout( xaxis_title=None,  height=500)
    fig_1.update_traces(texttemplate='%{text} %')

    # Feminicidios var
    feminicidios = filtered_fems.merge(filtered_data[['fk_euv','Edad Agresor']], left_on='fk_euv', right_on='fk_euv', how='left')
    feminicidios = feminicidios.drop_duplicates(keep='last')
    feminicidios = feminicidios.reset_index(drop=True)

    # Mapa violencias
    feminicidios_sent = feminicidios[feminicidios.Sentencia.str.contains("TENTATIVA")==False]
    fem_mun = feminicidios_sent.municipioV.value_counts()
    fem_mun = yuc[['geometry_mun','nom_mun']].merge(fem_mun, right_on=fem_mun.index, left_on='nom_mun', how='left')
    fem_mun.fillna(0,inplace=True)
    fem_mun.municipioV = fem_mun.municipioV.astype(int)

    fem_mun_global = fem_mun
    feminicidios_sent_global = feminicidios_sent

    merged = fem_mun_global.merge(feminicidios_sent_global[['EdadVictima','Edad Agresor','tipos','año_recepcion','TipoRelacion','pk_perfil_agresor','municipioV']], left_on='nom_mun', right_on='municipioV', how='left')
    #merged.fillna(0,inplace=True)
    merged = merged.replace(np.nan, '').astype(str)
    merged[merged.notnull()] = merged[merged.notnull()].astype(object)
    #merged.pk_perfil_agresor = merged.pk_perfil_agresor.astype(object)
    merged.drop(columns='municipioV_y',inplace=True)
    merged = merged[['nom_mun','EdadVictima','Edad Agresor','tipos','año_recepcion','TipoRelacion','pk_perfil_agresor']].groupby('nom_mun').sum()

    merged.TipoRelacion=merged.TipoRelacion.apply(lambda x: mayusculas(x))
    merged.tipos=merged.tipos.apply(lambda x: mayusculas(x))
    merged.EdadVictima=merged.EdadVictima.apply(lambda x: numeros(x))
    merged['Edad Agresor']=merged['Edad Agresor'].apply(lambda x: numeros(x))
    merged.pk_perfil_agresor=merged.pk_perfil_agresor.apply(lambda x: numeros(x))
    merged.año_recepcion=merged.año_recepcion.apply(lambda x: numeros(x))
    merged = merged.replace(',','', regex=True)


    merged.tipos = merged.tipos.str.split()
    merged.tipos = merged.tipos.apply(lambda x: unir_unicos(x))
    merged.EdadVictima = merged.EdadVictima.apply(lambda x: unir_resto(x))
    merged['Edad Agresor'] = merged['Edad Agresor'].apply(lambda x: unir_resto(x))
    merged.TipoRelacion = merged.TipoRelacion.apply(lambda x: unir_resto(x))
    merged.pk_perfil_agresor = merged.pk_perfil_agresor.apply(lambda x: unir_resto(x))
    merged.año_recepcion = merged.año_recepcion.apply(lambda x: unir_resto(x))

    test = merged.EdadVictima.apply(lambda x: x.split(', '))
    merged['len'] = test.apply(lambda x: len([int(i) for i in x if i.isdigit()==True]))
    test = test.apply(lambda x: sum([int(i) for i in x if i.isdigit()==True]))
    merged['EdadVictima'] = round(test.values/merged.len)

    test = merged['Edad Agresor'].apply(lambda x: x.split(', '))
    merged['len'] = test.apply(lambda x: len([int(i) for i in x if i.isdigit()==True]))
    test = test.apply(lambda x: sum([int(i) for i in x if i.isdigit()==True]))
    merged['Edad Agresor'] = round(test.values/merged.len)

    merged = merged.replace(np.nan, '').astype(str)
    merged = merged.replace('0.0', 'Sin datos').astype(str)
    #merged['EdadVictima'].values
    #merged.to_csv('test.csv', encoding='utf-8-sig')

    fem_mun_global = fem_mun_global.merge(merged, left_on='nom_mun', right_on=merged.index, how='left')
    fem_mun_global.fillna('', inplace=True)

    #fem_mun = fem_mun.merge(pob_fem[['NOM_MUN','POBFEM']], left_on='nom_mun', right_on='NOM_MUN', how='left')

    fem_mun_global['bins']=pd.cut(fem_mun_global["municipioV"],
        bins=[-1, 0.5, 1.5, 2.5,np.inf], 
        labels=['0','1','2','3 o más'])

    #fem_mun_global['nom_mun'] = fem_mun_global.set_index('nom_mun')
    
    
    # fg 2 - femnicidios por municipio treemap
    fig_2 = px.treemap(fem_mun_global,
                    path=[px.Constant("Yucatán"), 'nom_mun'],
                    values='municipioV',
                    hover_data={"municipioV":True,'EdadVictima':True,'tipos':True,'año_recepcion':True,'TipoRelacion':True,'pk_perfil_agresor':True, 'Edad Agresor':True},
                    labels = {'nom_mun':'Municipio', 'municipioV':'Víctimas','EdadVictima':'Edad Promedio Víctima','Edad Agresor':'Edad Promedio Agresor','tipos':'Violencias','año_recepcion':'Años','TipoRelacion':'Relación','pk_perfil_agresor':'pk agresor'},
                    title='Gráfica 2. Feminicidios Registrados por Municipio {}-{}'.format(min(filtered_data.año_recepcion),max(filtered_data.año_recepcion)),
                    #text='ola'
                    )
    
    fig_2.update_traces(hovertemplate='Víctimas=%{customdata[0]}<br>Edad Promedio Víctima=%{customdata[1]}<br>Edad Promedio Agresor=%{customdata[6]}<br>Violencias=%{customdata[2]}<br>Años=%{customdata[3]}')
    fig_2.update_traces(textinfo="label+value+percent root",)

    # fig 3 sunburst
    fem_agrav = filtered_fems[filtered_fems.Sentencia.isin(['FEMINICIDIO AGRAVADO','FEMINICIDIO'])].index
    filtered_fems.loc[fem_agrav,'Sentencia']='FEMINICIDIO'
    fem_agrav = filtered_fems[filtered_fems.Sentencia.isin(['FEMINICIDIO AGRAVADO EN GRADO DE TENTATIVA','FEMINICIDIO EN GRADO DE TENTATIVA'])].index
    filtered_fems.loc[fem_agrav,'Sentencia']='TENTATIVA DE FEMINICIDIO'
    sentencias = filtered_fems.Sentencia.value_counts()
    fig_3 = go.Figure(data=[go.Pie(labels=sentencias.index, values=sentencias.values, hole=.3 )])
    fig_3.update_layout(title='Gráfica 3. Porcentaje de Presuntos Feminicidios con Sentencia')
    fig_3.update_traces(textinfo="label+value+percent",showlegend=False, textposition='outside')

    # fig 4 - armas pie chart
    desc = filtered_data[filtered_data['Portaba Arma']=='Se desconoce'].index
    filtered_data.loc[desc,'Portaba Arma']='No'
    armas = filtered_data['Portaba Arma'].value_counts()
    fig_4 = go.Figure(data=[go.Pie(labels=armas.index, values=armas.values, hole=.3 )])
    fig_4.update_layout(title='Gráfica 4. Porcentaje de agresores armados')
    fig_4.update_traces(textinfo="label+value+percent",showlegend=False,)

    # fig 4 - armas treemap
    armas_mun = filtered_data[filtered_data['Portaba Arma']=='Sí']
    armas_mun = pd.DataFrame(armas_mun.groupby('municipiohechos')['Portaba Arma'].count())
    armas_mun = armas_mun.merge(yuc[['nom_mun','geometry_mun']], left_on=armas_mun.index, right_on='nom_mun', how='right')
    armas_mun.fillna(0, inplace=True)
    armas_mun['Portaba Arma'] = armas_mun['Portaba Arma'].astype(int)
    fig_5 = px.treemap(armas_mun,
                    path=[px.Constant("Yucatán"), 'nom_mun'],
                    values='Portaba Arma',
                    title='Gráfica 5. Municipios en los que los agresores han portado armas',
                    )

    fig_5.update_traces(hovertemplate='Municipio=%{label}<br>Portaban Arma=%{value}<br><extra></extra>')
    fig_5.update_traces(textinfo="label+value+percent root",)

    # Fig 6 - histo agresores
    edad_agresores = filtered_data[(filtered_data['Edad Agresor']>0)&(filtered_data['Edad Agresor']<99)]
    fig_6 = px.histogram(edad_agresores, x="Edad Agresor", color="Tipo de vínculo con victima", marginal="box",opacity=0.8,nbins=20,
                labels = {'Edad Agresor': 'Grupo Etario',},
                title = 'Gráfica 6. Distribución de agresores por vínculo con la víctima y edad'
                )
    fig_6.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 5
    )
    )

    # fig 7 - distribucion victimas
    fig_7 = px.histogram(victimas, x="Edad", color="Esta Embarazada", marginal="box",opacity=0.8,
            labels = {'Edad': 'Grupo Etario'},
            title = 'Gráfica 7. Distribución de casos por embarazo y edad',
            nbins=20
            )   
    fig_7.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 5
    )
    )

    return fig_1, fig_2, fig_3, fig_4, fig_5, fig_6, fig_7

In [ ]:
servicios_copia = servicios

In [ ]:
@app.callback( 
    Output('fig_8', "figure"),
    [
        Input("date-range", "start_date"),
        Input("date-range", "end_date")
    ], 
)
def update_servicios(start_date, end_date,):
    mask = (
        (servicios_copia.fecha_captura >= start_date)
        & (servicios_copia.fecha_captura <= end_date)
    )
    filtered_servicios = servicios_copia.loc[mask, :]
    servicios = pd.DataFrame(filtered_servicios[filtered_servicios.dependenciaquebrindoservicio.isna()==False])
    servicios_gropby = pd.DataFrame(servicios.groupby(['dependenciaquebrindoservicio','tiposervicio','serviciodetalle'])['numeroservicios'].count()).reset_index()
    fig_8 = px.sunburst(servicios_gropby, path=['dependenciaquebrindoservicio','tiposervicio','serviciodetalle'], values='numeroservicios', color='tiposervicio',
                    #labels={'labels':'Servicio Detalle'}, 
                    #hover_data={'id':False},
                    title='Gráfica 8. Distribución de los servicios por tipo e instancia capturista'
                    )
    print("plotly express hovertemplate:", fig.data[0].hovertemplate)
    fig_8.update_traces(hovertemplate='Servicio Detalle=%{label}<br>Número de Servicios=%{value}<br>Pertenencia=%{parent}<br><extra></extra>')
    fig_8.update_traces(textinfo="label+value+percent entry",)

    return fig_8

In [ ]:
@app.callback( 
    Output('map_tasa', "figure"),
    [
        Input("tasa", "value"),
        Input("violencia-tipo", "value")
    ], 
)
def update_tasa(year, violencia):
    mask = (
        (victimas.año_recepcion == year)
    )
    tasa_mask = (victimas.año_recepcion == year)

    filtered_data = victimas.loc[mask, :]
    filtered_tasa = victimas.loc[tasa_mask, :]

    # conapo year
    pob_fem = (conapo_copia[(conapo_copia['CLAVE_ENT']==31)&(conapo_copia['AÑO']==year)&(conapo_copia['SEXO']=='Mujeres')]).reset_index(drop=True)
    pob_fem = pob_fem[['MUN','POB']]
    pob_fem = pob_fem.groupby('MUN').sum()

    # Tipos de violencia
    tipos = filtered_data.iloc[:,[i for i in range(42,49)]]
    tipos = pd.DataFrame(tipos.sum())
    tipos = tipos.sort_values(0,ascending=False)
    
    # Mapa tasas violencias
    victimas_2020 = filtered_tasa
    casos_mun = victimas_2020.groupby('municipiohechos')['fk_euv'].count()
    casos_mun = yuc[['geometry_mun','nom_mun']].merge(casos_mun, right_on=casos_mun.index, left_on='nom_mun', how='left')
    casos_mun.fillna(0,inplace=True)
    casos_mun = pd.DataFrame(casos_mun.merge(pob_fem,left_on='nom_mun',right_on='MUN',how='left'))
    casos_mun[['fk_euv','POB']] = casos_mun[['fk_euv','POB']].astype(int)

    mun_violencias = victimas_2020.groupby('municipiohechos')[['Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro']].sum()
    mun_violencias = yuc[['geometry_mun','nom_mun']].merge(mun_violencias, right_on=mun_violencias.index, left_on='nom_mun', how='left')
    mun_violencias.fillna(0, inplace=True)

    mun_violencias = pd.DataFrame(mun_violencias.merge(pob_fem,left_on='nom_mun',right_on='MUN',how='left'))

    if violencia != 'Tasa 1000 Total':
        mun_violencias['{} Tasa 1000'.format(violencia)] = round(mun_violencias[violencia]/mun_violencias['POB'].astype(int)*1000,2) # added
        mun_violencias['{}_casos'.format(violencia)] = mun_violencias.apply(lambda row: int(row[violencia]), axis=1) # added

    mun_violencias = mun_violencias.merge(casos_mun[['fk_euv','nom_mun']], left_on=mun_violencias.index, right_on=casos_mun.index, how='left')
    mun_violencias[['fk_euv','POB']] = mun_violencias[['fk_euv','POB']].astype(int)
    mun_violencias['Tasa 1000 Total'] =round(mun_violencias.fk_euv/mun_violencias.POB*1000,2)
    mun_violencias.drop(['key_0','nom_mun_y'],axis='columns', inplace=True)
    #mun_violencias['text_total'] = mun_violencias.apply(lambda row: f"Casos Totales: {int(row['fk_euv'])}<br>POBFEM: {row['POBFEM']}", axis=1)

    gdf = geopandas.GeoDataFrame(mun_violencias, geometry='geometry_mun')
    gdf = gdf.set_index('nom_mun_x')
    gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))

    #cols_dd = ['Tasa 1000 Total','Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro',]
    # we need to add this to select which trace 
    # is going to be visible
    #visible = np.array(cols_dd)

    poly_json = json.loads(gdf.geometry_mun.to_json())

    # define traces and buttons at once
    traces = []
    #buttons = []

    # Agregar mapa general
    traces.append(go.Choroplethmapbox(
            geojson=poly_json,
            locations=gdf.index, # Spatial coordinates
            z=gdf[violencia], # Data to be color-coded
            colorbar_title='Tasa',
            name='Tasa {}'.format(year),
            #hoverlabel={'Tasa 1000 Total':False},
            #hovertemplate = "Mun_x: %{z}<br>Ola: %{customdata[0]}<br>Fin: %{customdata[-1]}",
            #text = gdf['text_total'], # hover text
            text = gdf.index, # hover text
            customdata=np.stack([ gdf['POB'], gdf['fk_euv']], axis=-1),
            hovertemplate='<b>%{text}</b>'+ '<br>' +'Tasa: ' + '%{z}' +  '<br>' +'Casos Totales: ' + '%{customdata[1]}' +'<br>Población Femenina: ' + '%{customdata[0]}',
            #visible= True if 'Tasa 1000 Total'==cols_dd[0] else False,
            ))

    # Show figure
    map_tasa = go.Figure(data=traces,
                    #layout=dict(updatemenus=updatemenus)
                    )
    # This is in order to get the first title displayed correctly
    #first_title = cols_dd[0]
    map_tasa.update_layout(title=f"Mapa 1. Tasa de casos de violencia por cada 1000 mujeres",)
    map_tasa.update_geos(fitbounds="locations", visible=False, )
    if violencia != 'Tasa 1000 Total':
        map_tasa.add_annotation(
            # The arrow head will be 25% along the x axis, starting from the left
            x=0,
            # The arrow head will be 40% along the y axis, starting from the bottom
            y=0.98,
            text="<b>Temporalidad:</b> {}<br><b>Total de casos:</b> {}".format(min(victimas_2020.año_recepcion),int(mun_violencias[violencia].sum())),
            showarrow=False,
            bordercolor="black",
            bgcolor="white",
            borderwidth=1.5,
            opacity=0.8
        )
    else:
        map_tasa.add_annotation(
            # The arrow head will be 25% along the x axis, starting from the left
            x=0,
            # The arrow head will be 40% along the y axis, starting from the bottom
            y=0.98,
            text="<b>Temporalidad:</b> {}<br><b>Total de casos:</b> {}".format(min(victimas_2020.año_recepcion),len(victimas_2020)),
            showarrow=False,
            bordercolor="black",
            bgcolor="white",
            borderwidth=1.5,
            opacity=0.8
        )

    map_tasa.update_layout(mapbox_style="carto-positron",
                    mapbox_zoom=7.5, mapbox_center = {"lat": 20.400417, "lon": -89.134857})

    return map_tasa

In [ ]:
@app.callback( 
    Output('map_1', "figure"),
    [
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
        Input("sentencia-tipo", "value"),
    ], 
)
def update_fems(start_date, end_date, sentencia):
    mask = (
        (victimas.fecha_recepcion >= start_date)
        & (victimas.fecha_recepcion <= end_date)
    )
    feminicidios = feminicidios_copia
    fem_mask = (
        (feminicidios.fecha_recepcion >= start_date)
        & (feminicidios.fecha_recepcion <= end_date)
    )
    

    filtered_data = victimas.loc[mask, :]
    filtered_fems = feminicidios.loc[fem_mask, :]

    # Tipos de violencia
    tipos = filtered_data.iloc[:,[i for i in range(42,49)]]
    tipos = pd.DataFrame(tipos.sum())
    tipos = tipos.sort_values(0,ascending=False)


    # Feminicidios var
    feminicidios = filtered_fems.merge(filtered_data[['fk_euv','Edad Agresor']], left_on='fk_euv', right_on='fk_euv', how='left')
    feminicidios = feminicidios.drop_duplicates(keep='last')
    feminicidios = feminicidios.reset_index(drop=True)

    # Mapa violencias
    feminicidios_sent = feminicidios[feminicidios.Sentencia.str.contains("TENTATIVA")==False]
    tentativas_sent = feminicidios[feminicidios.Sentencia.str.contains("TENTATIVA")==True]
    fem_mun = feminicidios_sent.municipioV.value_counts()
    fem_mun = yuc[['geometry_mun','nom_mun']].merge(fem_mun, right_on=fem_mun.index, left_on='nom_mun', how='left')
    fem_mun.fillna(0,inplace=True)
    fem_mun.municipioV = fem_mun.municipioV.astype(int)

    fem_mun_tent = tentativas_sent.municipioV.value_counts()
    fem_mun_tent = yuc[['geometry_mun','nom_mun']].merge(fem_mun_tent, right_on=fem_mun_tent.index, left_on='nom_mun', how='left')
    fem_mun_tent.fillna(0,inplace=True)
    fem_mun_tent.municipioV = fem_mun_tent.municipioV.astype(int)

    if sentencia == 'Feminicidios':
        fem_mun_global = fem_mun
        feminicidios_sent_global = feminicidios_sent
    else:
        fem_mun_global = fem_mun_tent
        feminicidios_sent_global = tentativas_sent


    merged = fem_mun_global.merge(feminicidios_sent_global[['EdadVictima','Edad Agresor','tipos','año_recepcion','TipoRelacion','pk_perfil_agresor','municipioV']], left_on='nom_mun', right_on='municipioV', how='left')
    #merged.fillna(0,inplace=True)
    merged = merged.replace(np.nan, '').astype(str)
    merged[merged.notnull()] = merged[merged.notnull()].astype(object)
    #merged.pk_perfil_agresor = merged.pk_perfil_agresor.astype(object)
    merged.drop(columns='municipioV_y',inplace=True)
    merged = merged[['nom_mun','EdadVictima','Edad Agresor','tipos','año_recepcion','TipoRelacion','pk_perfil_agresor']].groupby('nom_mun').sum()

    merged.TipoRelacion=merged.TipoRelacion.apply(lambda x: mayusculas(x))
    merged.tipos=merged.tipos.apply(lambda x: mayusculas(x))
    merged.EdadVictima=merged.EdadVictima.apply(lambda x: numeros(x))
    merged['Edad Agresor']=merged['Edad Agresor'].apply(lambda x: numeros(x))
    merged.pk_perfil_agresor=merged.pk_perfil_agresor.apply(lambda x: numeros(x))
    merged.año_recepcion=merged.año_recepcion.apply(lambda x: numeros(x))
    merged = merged.replace(',','', regex=True)


    merged.tipos = merged.tipos.str.split()
    merged.tipos = merged.tipos.apply(lambda x: unir_unicos(x))
    merged.EdadVictima = merged.EdadVictima.apply(lambda x: unir_resto(x))
    merged['Edad Agresor'] = merged['Edad Agresor'].apply(lambda x: unir_resto(x))
    merged.TipoRelacion = merged.TipoRelacion.apply(lambda x: unir_resto(x))
    merged.pk_perfil_agresor = merged.pk_perfil_agresor.apply(lambda x: unir_resto(x))
    merged.año_recepcion = merged.año_recepcion.apply(lambda x: unir_resto(x))

    test = merged.EdadVictima.apply(lambda x: x.split(', '))
    merged['len'] = test.apply(lambda x: len([int(i) for i in x if i.isdigit()==True]))
    test = test.apply(lambda x: sum([int(i) for i in x if i.isdigit()==True]))
    merged['EdadVictima'] = round(test.values/merged.len)

    test = merged['Edad Agresor'].apply(lambda x: x.split(', '))
    merged['len'] = test.apply(lambda x: len([int(i) for i in x if i.isdigit()==True]))
    test = test.apply(lambda x: sum([int(i) for i in x if i.isdigit()==True]))
    merged['Edad Agresor'] = round(test.values/merged.len)

    merged = merged.replace(np.nan, '').astype(str)
    merged = merged.replace('0.0', 'Sin datos').astype(str)
    #merged['EdadVictima'].values
    #merged.to_csv('test.csv', encoding='utf-8-sig')

    fem_mun_global = fem_mun_global.merge(merged, left_on='nom_mun', right_on=merged.index, how='left')
    fem_mun_global.fillna('', inplace=True)

    #fem_mun = fem_mun.merge(pob_fem[['NOM_MUN','POBFEM']], left_on='nom_mun', right_on='NOM_MUN', how='left')

    fem_mun_global['bins']=pd.cut(fem_mun_global["municipioV"],
        bins=[-1, 0.5, 1.5, 2.5,np.inf], 
        labels=['0','1','2','3 o más'])

    gdf = geopandas.GeoDataFrame(fem_mun_global, geometry='geometry_mun')
    gdf = gdf.set_index('nom_mun')
    gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))

    # colores
    colors = px.colors.qualitative.Prism[4:8]


    poly_json = json.loads(gdf.geometry_mun.to_json())

    # define traces and buttons at once
    traces = []
    

    # Agregar mapa general
    traces.append(go.Choroplethmapbox(
            geojson=poly_json,
            locations=gdf.index, # Spatial coordinates
            z=gdf['municipioV'], # Data to be color-coded
            colorscale=[[0, colors[0]],[0.25, colors[0]],
            [0.25, colors[1]],[0.5, colors[1]],
            [0.5, colors[2]],[0.75, colors[2]],
            [0.75, colors[3]], [1, colors[3]],],
            zmin=0, zmax=3,
            colorbar=dict(
            title="Núm. Víctimas<br>",
            titleside="top",
            tickmode="array",
            tickvals=[0.375, 1.125, 1.875, 2.625],
            ticktext=["0", "1", "2", "3 o más"],
            ticks="outside"
            ),
            #colorbar_title='Casos',
            name=sentencia,
            #hoverlabel={'Tasa 1000 Total':False},
            #hovertemplate = "Mun_x: %{z}<br>Ola: %{customdata[0]}<br>Fin: %{customdata[-1]}",
            #text = gdf['text_total'], # hover text
            text = gdf.index, # hover text
            customdata=np.stack([gdf['TipoRelacion'], gdf['EdadVictima'], gdf['Edad Agresor'], gdf['tipos'],gdf['año_recepcion'], gdf['TipoRelacion']], axis=-1),
            hovertemplate='<b>%{text}</b>'+ '<br>' +'Número de Víctimas: ' + '%{z}' +  '<br>' +'Promedio Edad Víctimas: ' 
            + '%{customdata[1]}' +'<br>Promedio Edad Agresores: ' + '%{customdata[2]}'+'<br>Tipos de violencia: ' + '%{customdata[3]}'
            +'<br>Año de recepción del caso: ' + '%{customdata[4]}' + '<br>Vínculo con el agresor: ' + '%{customdata[5]}',
            ))

    # Show figure
    fig = go.Figure(data=traces,
                    )
    # This is in order to get the first title displayed correctly
    #first_title = cols_dd[0]
    fig.update_layout(title="Mapa 2. {} Registrados por Municipio".format(sentencia))
    fig.update_geos(fitbounds="locations", visible=False, )
    fig.add_annotation(
        # The arrow head will be 25% along the x axis, starting from the left
        x=0,
        # The arrow head will be 40% along the y axis, starting from the bottom
        y=0.98,
        text="<b>Temporalidad:</b> {}-{}<br><b>Femincidios:</b> {}<br><b>Tentativas:</b> {}".format(start_date[:4],end_date[:4],(len(feminicidios_sent)),(len(tentativas_sent))),
        showarrow=False,
        bordercolor="black",
        bgcolor="white",
        borderwidth=1.5,
        opacity=0.8
    )

    fig.update_layout(mapbox_style="carto-positron",
                    mapbox_zoom=7.5, mapbox_center = {"lat": 20.400417, "lon": -89.134857})
    
    return fig

In [ ]:
@app.callback( 
    Output('map_2', "figure"),
    [
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ], 
)
def update_armas(start_date, end_date):
    mask = (
        (victimas.fecha_recepcion >= start_date)
        & (victimas.fecha_recepcion <= end_date)
    )
    filtered_data = victimas.loc[mask, :]
    armas_mun = filtered_data[filtered_data['Portaba Arma']=='Sí']
    armas_mun = pd.DataFrame(armas_mun.groupby('municipiohechos')['Portaba Arma'].count())
    armas_mun = armas_mun.merge(yuc[['nom_mun','geometry_mun']], left_on=armas_mun.index, right_on='nom_mun', how='right')
    armas_mun.fillna(0, inplace=True)
    armas_mun['Portaba Arma'] = armas_mun['Portaba Arma'].astype(int)
    gdf = geopandas.GeoDataFrame(armas_mun, geometry='geometry_mun')
    gdf = gdf.set_index('nom_mun')
    gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))
    gdf['cat']=pd.cut(gdf['Portaba Arma'],
        bins=[-1, 0.9, 10.9,20.9,35.9,np.inf], 
        labels=['0','1-10','11-20','21-35','36 o más']
        #labels=[1,2,3,4,5]
        )
    gdf = gdf.sort_values('Portaba Arma')
    colors = px.colors.qualitative.Prism[3:9]
    map_2 = px.choropleth_mapbox(gdf, geojson=gdf.geometry_mun, 
                    locations=gdf.index, color='cat',
                    color_discrete_sequence=colors,
                    labels = {'nom_mun':'Municipio', 'Portaba Arma':'Agresores Armados','cat':'Agresores<br>Armados'},
                    hover_data={'Portaba Arma':True},
                    #title='Mapa 2. Feminicidios Registrados por Municipio',
                    #color_continuous_midpoint=np.average(gdf['municipioV'], weights=gdf['POBFEM'].astype(int)),
                    center={"lat": 20.400417, "lon": -89.134857},
                    mapbox_style="carto-positron", 
                    zoom=7.5,
                    #opacity=0.5,
                    title='Mapa 3. Agresores Armados por Municipio',
                    )
    map_2.update_geos(fitbounds="locations", visible=False)

    #fig.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

    map_2.add_annotation(
        # The arrow head will be 25% along the x axis, starting from the left
        x=0,
        # The arrow head will be 40% along the y axis, starting from the bottom
        y=0.98,
        text="<b>Temporalidad:</b> {}-{}<br><b>Total de casos:</b> {}".format(min(filtered_data.año_recepcion),max(filtered_data.año_recepcion),len(filtered_data)),
        showarrow=False,
        bordercolor="black",
        bgcolor="white",
        borderwidth=1.5,
        opacity=0.8
    )

    map_2.update(layout = dict(title=dict(x=0.5)))
    map_2.update_layout(title_y=1, title_x=0)
    #print(fig.data[0].hovertemplate)
    map_2.update_traces(hovertemplate='Municipio=%{location}<br>Agresores Armados=%{customdata[0]}<extra></extra>')

    return map_2

In [ ]:
@app.callback( 
    Output('map_3', "figure"),
    [
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ], 
)
def update_edades(start_date, end_date):
    mask = (
        (victimas.fecha_recepcion >= start_date)
        & (victimas.fecha_recepcion <= end_date)
    )
    filtered_data = victimas.loc[mask, :]

    mun_edad = pd.DataFrame(filtered_data.groupby('municipiohechos')['Edad'].mean().round().astype(int)) # ya que es una distribucion normal, se puede usar la media como agregacion
    mun_edad = mun_edad.rename(columns={'Edad':'Edad Promedio'})

    mun_edad = mun_edad.merge(yuc[['nom_mun','geometry_mun']], left_on=mun_edad.index, right_on='nom_mun', how='right')
    mun_edad.fillna(0, inplace=True)
    gdf = geopandas.GeoDataFrame(mun_edad, geometry='geometry_mun')
    gdf = gdf.set_index('nom_mun')
    gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))
    map_3 = px.choropleth_mapbox(gdf, geojson=gdf.geometry_mun, 
                        locations=gdf.index, color="Edad Promedio",
                        color_continuous_scale="Viridis",
                        labels={'nom_mun': 'Municipio'},
                        #labels = {'nom_mun':'Municipio', 'municipioV':'Víctimas','EdadVictima':'Edades','tipos':'Violencias','año_recepcion':'Años','TipoRelacion':'Relación','pk_perfil_agresor':'pk agresor'},
                        #hover_data={"municipioV":True,'EdadVictima':True,'tipos':True,'año_recepcion':True,'TipoRelacion':True,'pk_perfil_agresor':True,'Hubo Feminicidio':False },
                        title='Mapa 4. Edad Promedio de las Víctimas por Municipio',
                        #color_continuous_midpoint=np.average(victimas['Edad']),
                        center={"lat": 20.400417, "lon": -89.134857},
                        mapbox_style="carto-positron", 
                        zoom=7.5,
                        #opacity=0.5,
                        )
    map_3.update_geos(fitbounds="locations", visible=False)

    #fig.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

    map_3.add_annotation(
        # The arrow head will be 25% along the x axis, starting from the left
        x=0,
        # The arrow head will be 40% along the y axis, starting from the bottom
        y=0.98,
        text="<b>Temporalidad:</b> {}-{}<br><b>Total de casos:</b> {}".format(min(filtered_data.año_recepcion),max(filtered_data.año_recepcion),len(filtered_data)),
        showarrow=False,
        bordercolor="black",
        bgcolor="white",
        borderwidth=1.5,
        opacity=0.8
    )

    map_3.update(layout = dict(title=dict(x=0.5)))
    map_3.update_layout(
        margin={"r":0,"t":30,"l":10,"b":10},
        coloraxis_colorbar={
            'title':'Edad'})
    map_3.update_layout(title_y=1, title_x=0)
    map_3.update_layout(coloraxis_showscale=True)

    return map_3

In [ ]:
if __name__ == "__main__":
    app.run_server(debug=True, use_reloader=False)